In [1]:
import nosbench
from nosbench.program import Program, Instruction, Pointer
from nosbench.function import Function, interpolate, bias_correct, clip, size
import torch

import pprint

from neps.space.new_space import space
from neps.space.new_space import config_string

EPOCHS_PER_CONFIG = 1
AVAILABLE_VARIABLE_SLOTS = 11 # As per the NOSBench paper

In [2]:
import nosbench
from nosbench.program import Program, Instruction, Pointer
from nosbench.function import Function, interpolate, bias_correct, clip, size
from nosbench.utils import prune_program
import torch

import pprint
import math

from neps.space.new_space import space

from functools import partial

import numpy as np
import pytest

import neps
import neps.space.new_space.space as space
import neps.space.new_space.bracket_optimizer as new_bracket_optimizer

MAX_EPOCHS_PER_CONFIG = 20          # As per the NOSBench paper
AVAILABLE_VARIABLE_SLOTS = 11       # As per the NOSBench paper


def _function_writer(operation: callable, *args, store: int) -> tuple[Function, list[Pointer], Pointer]:
    return Function(operation, len(args)), [Pointer(var) if isinstance(var, int) else Pointer(var()) for var in args], store
    
def program_compiler(instruction: tuple[callable, list[Pointer], Pointer], *args) -> Program:
    if len(args) == 0:
        return Program([Instruction(instruction[0], inputs=instruction[1], output=instruction[2])])
    else:
        return Program(program_compiler(instruction) + sum(args,[]))

class nosBench(space.Pipeline):
    _UNARY_FUN = space.Categorical(
        choices=(torch.square, torch.exp, torch.log, torch.sign, torch.sqrt, torch.abs, torch.norm, torch.sin, torch.cos, torch.tan, torch.asin, torch.acos, torch.atan, torch.mean, torch.std, size,)
    )

    _BINARY_FUN = space.Categorical(
        choices=(clip, torch.div, torch.mul, torch.add, torch.sub, torch.minimum, torch.maximum, torch.heaviside)
    )

    _TERNARY_FUN = space.Categorical(
        choices=(interpolate, bias_correct,)
    )

    _CONST = space.Integer(3, 8) # 0-2 are reserved for weights, gradient, and optimizer step
    _VAR = space.Integer(9, AVAILABLE_VARIABLE_SLOTS+9) # And 3-8 are reserved for the constants

    # Is this the right way to handle constants?
    _PARAMS = space.Categorical(
        choices=(
            0, # weights
            1, # gradient
            2, # optimizer step
        )
    )

    _POINTER = space.Categorical(
        choices=(
            space.Resampled(_PARAMS),
            space.Resampled(_CONST),
            space.Resampled(_VAR),
        ),
    )

    _F_ARGS = space.Categorical(
        choices=(
            (
                space.Resampled(_UNARY_FUN),
                space.Resampled(_POINTER),
            ),
            (
                space.Resampled(_BINARY_FUN),
                space.Resampled(_POINTER),
                space.Resampled(_POINTER),
            ),
            (
                space.Resampled(_TERNARY_FUN),
                space.Resampled(_POINTER),
                space.Resampled(_POINTER),
                space.Resampled(_POINTER),
            ),
        ),
    )

    _F = space.Operation(
        operator=_function_writer,
        args=space.Resampled(_F_ARGS),
        kwargs={"store": space.Resampled(_VAR)}, # move to _F_ARGS as normal arg?
    )

    _P_ARGS = space.Categorical(
        choices=(
            (space.Resampled(_F),),
            (space.Resampled(_F), space.Resampled("program"),), # Can be multiple space.Resampled("P")
        ),
        prior_index=1, # Encourages longer programs
        prior_confidence=space.ConfidenceLevel.LOW
    )

    program = space.Operation(
        operator=program_compiler,
        args=space.Resampled(_P_ARGS),
    )

    # epochs = space.Fidelity(space.Integer(1, MAX_EPOCHS_PER_CONFIG))


In [5]:
pipeline = nosBench()

resolved_pipeline, resolution_context = space.resolve(pipeline)

p = resolved_pipeline.program
p_config_string = space.convert_operation_to_string(p)
assert p_config_string 
pretty_config = config_string.ConfigString(p_config_string).pretty_format()
assert pretty_config

# print()
# print("Config string:")
# print(pretty_config)

program = space.convert_operation_to_callable(p)
pprint.pprint(program)

benchmark = nosbench.ToyBenchmark()
fitness = benchmark.query(program, EPOCHS_PER_CONFIG)
print(fitness)


[Instruction(Function(interpolate, 3), inputs=[0, 0, 7], output=16),
 Instruction(Function(size, 1), inputs=[10], output=16),
 Instruction(Function(sin, 1), inputs=[19], output=15),
 Instruction(Function(heaviside, 2), inputs=[19, 3], output=12)]
1.1094942728678385


In [16]:
SGD = Program(
    [
        # momentum = 1 - 0.1 = 0.9
        Instruction(Function(torch.sub, 2), [Pointer(3), Pointer(5)], Pointer(9)),
        # m = m * momentum
        Instruction(Function(torch.mul, 2), [Pointer(10), Pointer(9)], Pointer(10)),
        # m = gradient + m
        Instruction(Function(torch.add, 2), [Pointer(1), Pointer(10)], Pointer(10)),

        Instruction(Function(torch.div, 2), [Pointer(10), Pointer(4)], Pointer(16)), # Unnecessary line, should be removed by pruning
        # update = m * 0.001
        Instruction(Function(torch.mul, 2), [Pointer(10), Pointer(6)], Pointer(11)),
    ]
)
pprint.pprint(SGD)
prune_program(SGD)
pprint.pprint(SGD)

# from nosbench.optimizers import AdamW
# for e in range(20):
#     benchmark = nosbench.NOSBench()
#     loss = benchmark.query(AdamW, epoch=e)
#     print(loss)
    

[Instruction(Function(sub, 2), inputs=[3, 5], output=9),
 Instruction(Function(mul, 2), inputs=[10, 9], output=10),
 Instruction(Function(add, 2), inputs=[1, 10], output=10),
 Instruction(Function(div, 2), inputs=[10, 4], output=16),
 Instruction(Function(mul, 2), inputs=[10, 6], output=11)]
[Instruction(Function(sub, 2), inputs=[3, 5], output=9),
 Instruction(Function(mul, 2), inputs=[10, 9], output=10),
 Instruction(Function(add, 2), inputs=[1, 10], output=10),
 Instruction(Function(mul, 2), inputs=[10, 6], output=11)]


In [14]:
import time

t0 = time.perf_counter()
samples = 100

for _ in range(samples):
    resolved_pipeline, resolution_context = space.resolve(pipeline)

t1 = time.perf_counter()

print(f"sampling takes {(t1 - t0) / samples:.2}s on average over {samples} samples")
print(f"duration for {samples} samples: {t1 - t0:.3}s ")

sampling takes 0.0018s on average over 100 samples
duration for 100 samples: 0.183s 
